# 2 Galeano, Joseph & Lillo (2015)

# 3

### 3.b.2 


_Berrendero, J. R., Justel, A., & Svarc, M. (2011)_

Sean los p procesos, cada uno en el **mismo intervalo T**:

$$\chi_1, \dots, \chi_p$$

Note que al tomar el punto $t\in T$, se tiene un vector:

$$\chi_1(t) = [X_{11}(t), \dots, X_{1n}(t)]^T$$ 

Por ellos se define la matriz 

$$\begin{bmatrix}\chi_1(t) | \chi_2(t) | \dots | \chi_p(t)\end{bmatrix}$$ 

Y se toman la matriz de covarianza de la matriz anterior, la cual se denota como:

$$\Sigma(t) = \begin{bmatrix} \Sigma_{11}(t) & \Sigma_{12}(t) & \dots & \Sigma_{1p}(t) \\ \Sigma_{21}(t) & \Sigma_{22}(t) & \dots & \Sigma_{2p}(t) \\ \vdots & \vdots & \ddots & \vdots \\ \Sigma_{p1}(t) & \Sigma_{p2}(t) & \dots & \Sigma_{pp}(t) \end{bmatrix}$$

Pero tomando todos los puntos, se tiene la sucesión de matrices de covarianza:

$$\Sigma_1, \Sigma_2, \dots, \Sigma_T$$

Tomando los valores propios y vectores propios de cada matriz de covarianza, se tiene que para cada $t\in T$:

$$\Sigma_t = \lambda(t) V_t $$

Donde $\lambda(t)$ es una **función** y loz vectores $V_t$ son dependientes de $t$. La idea es unir los vectores en un vector de funciones.


Cual multiplo de los vectores propios es vector propio, es decir, si $V_t$ es vector propio, entonces $cV_t$ también lo es, mostrando que al ser multiplicado **por un c negativo cambia de dirección**, haciendo que no sea unico.

Medida de varianza explicada para toda la parte del proceso para que no hayas problemas.


# 4 Imagenes

Se puede realizar la descomposición 

$$\chi(i, j) = \sum_{k = 1}^{\infty} c_k u_k(j, l)$$

Tal que la función base se puede definir como:
$$u_k(j, l) =u_k(j) u_k(l)$$

In [2]:
import rpy2

In [4]:
%load_ext rpy2.ipython

In [5]:
%%R
install.packages("funData")
install.packages("MFPCA")

* installing *source* package ‘dotCall64’ ...
** package ‘dotCall64’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0’
using Fortran compiler: ‘GNU Fortran (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0’


gcc -I"/usr/share/R/include" -DNDEBUG      -fopenmp -I../inst/include/ -DDOTCAL64_PRIVATE -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c dotCall64.c -o dotCall64.o
gcc -I"/usr/share/R/include" -DNDEBUG      -fopenmp -I../inst/include/ -DDOTCAL64_PRIVATE -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c dotCall64helpers.c -o dotCall64helpers.o
gcc -I"/usr/share/R/include" -DNDEBUG      -fopenmp -I../inst/include/ -DDOTCAL64_PRIVATE -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c init.c -o init.o
gcc -I"/usr/share/R/include" -DNDEBUG      -fopenmp -I../inst/include/ -DDOTCAL64_PRIVATE -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -

installing to /usr/local/lib/R/site-library/00LOCK-dotCall64/00new/dotCall64/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (dotCall64)
* installing *source* package ‘maps’ ...
** package ‘maps’ successfully unpacked and MD5 sums checked
** using staged installation


checking for gawk... no
checking for mawk... mawk
configure: creating ./config.status
config.status: creating src/Makefile


** libs


...

config.status: creating src/Makevars
gcc -I"/usr/share/R/include" -DNDEBUG       -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c MFPCA-init.c -o MFPCA-init.o


** libs
using C compiler: ‘gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0’


gcc -I"/usr/share/R/include" -DNDEBUG       -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c dct2D.c -o dct2D.o
gcc -I"/usr/share/R/include" -DNDEBUG       -fpic  -g -O2 -ffile-prefix-map=/build/r-base-6tgf7J/r-base-4.4.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c dct3D.c -o dct3D.o
gcc -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -flto=auto -ffat-lto-objects -flto=auto -Wl,-z,relro -o MFPCA.so MFPCA-init.o dct2D.o dct3D.o -L/usr/lib/R/lib -lR


installing to /usr/local/lib/R/site-library/00LOCK-MFPCA/00new/MFPCA/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (MFPCA)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘dotCall64’, ‘spam’, ‘maps’, ‘fields’

probando la URL 'https://cloud.r-project.org/src/contrib/dotCall64_1.2.tar.gz'
Content type 'application/x-gzip' length 20362 bytes (19 KB)
downloaded 19 KB

probando la URL 'https://cloud.r-project.org/src/contrib/spam_2.11-1.tar.gz'
Content type 'application/x-gzip' length 1812654 bytes (1.7 MB)
downloaded 1.7 MB

probando la URL 'https://cloud.r-project.org/src/contrib/maps_3.4.2.1.tar.gz'
Content type 'application/x-gzip' length 2273638 bytes (2.2 MB)
downloaded 2.2 MB

probando la URL 'https://cloud.r-project.org/src/contrib/fields_16.3.tar.gz'
Content type 'application/x-gzip' length 3623407 bytes (3.5 MB)
downloaded 3.5 MB

probando la URL 'https://cloud.r-project.org/src/contrib/funData_1.3-9.tar.gz'
Content type 'application/x-gzip' length 112619 bytes (109 KB)
downloaded 109 KB


The downloaded source packages are in
	‘/tmp/